# Item-Based Nearest Neighbor Collaborative Filtering
## MovieLens Dataset

최근접 이웃 협업 필터링은 사용자 기반과 아이템 기반으로 나뉘는데, 이 중 일반적으로 아이템 기반의 추천 정확도가 뛰어나므로 아이템 기반 방식을 이용해 추천 시스템 구현 실습을 수행해보겠다.
협업 필터링 기반의 영화 추천을 위해서는 사용자가 영화에 대해 작성한 평점 데이터 행렬 세트가 필요한데, 이를 위해 Grouplens의 MovieLens 데이터셋을 사용하도록 하겠다.

https://grouplens.org/datasets/movielens/latest/

### - ml-latest-small.zip (size: 1 MB)

100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018.

- links.csv
- movies.csv
- ratings.csv
-tags.csv

----

## Data Loading and Processing

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
movies = pd.read_csv('./data/MovieLens_Dataset/ml-latest-small/movies.csv')
ratings = pd.read_csv('./data/MovieLens_Dataset/ml-latest-small/ratings.csv')

print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [3]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [4]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


- DataFrame의 pivot_table() 함수 이용하여 ratings 데이터프레임을 User-Items 행렬로 변환

In [5]:
ratings = ratings[['userId', 'movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index='userId', columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- pivot_table() 적용 이후 movieId 값이 모두 column 이름으로 변경되었다.
- NaN 값이 많은 Sparse matrix인데, 편의상 NaN은 모두 0점으로 변환하도록 하자. (최소 평점 0.5)
- 가독성을 위해 column 명을 movieId에서 title로 변경하자. (title은 movies dataset에 존재)

In [7]:
# title column을 얻기 위한 movies와 join
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title'로 title columnㅇ로 pivot 수행
ratings_matrix = rating_movies.pivot_table('rating',index='userId', columns='title')

# NaN -> 0
ratings_matrix = ratings_matrix.fillna(0)

ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Get Simmilarities between Movies

변환된 User-Item 행렬을 이용하여 영화 간의 유사도를 측정해보자.
영화 간의 유사도는 이전에서와 같이 코사인 유사도를 사용하겠다.
코사인 유사도를 구하기 위한 함수인 cosine_similarity()는 행을 기준으로 서로 다른 행을 비교해 유사도를 산출하므로, 영화를 기준으로 코사인 유사도를 계산하기 위해 ratings_matrix의 행과 열을 바꿔 전치행렬을 만들어준다.

In [8]:
ratings_matrix_T = ratings_matrix.T
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ratings_matrix의 전치행렬을 구했으니, 이를 기반으로 코사인 유사도를 구해보자.
좀 더 직관적인 영화의 유사도 값을 표현하기 위해 cosine_similarity()로 반환된 Numpy 행렬에 영화명을 mapping해 DataFrame으로 변환해보자.

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# Numpy matrix to DataFrame
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,\
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


- item_sim_df를 이용해서 '대부(Godfather, The (1972))'와 유사도가 높은 상위 5개의 영화를 추출해보자. (자신 제외)

In [11]:
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[1:6]

title
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

## Personalized Movie Recommend System Using Item-based NNCF

앞에서 만들어진 아이템 기잔 영화 유사도 데이터는 모든 사용자의 평점을 기준으로 영화의 유사도를 생성했고, 이를 이용해 추천을 진행하였다. 나름 훌룡한 성능을 보이긴 했으나 개인적인 취향을 반영하지 않았기 때문에 추천 시스템으로 사용하기에는 다소 부족함이 있다.
이번에는 최근접 이웃 방식을 적용해 개인 취향을 고려한 추천 시스템의 구축을 진행해보자. 개인화된 영화 추천의 가장 큰 특징은 개인이 아직 관람하지 않은 영화를 추천한다는 점으로, 아직 관람하지 않은 영화의 아이템 유사도와 관람한 영화의 평점 데이터를 기반으로 새롭게 예측 평점을 계산하고 추천을 진행한다.

이러한 아이템 기반의 협업 필터링에서 개인화된 예측 평점을 구하는 공식은 다음과 같다.

${\hat R}_{\mu, i} = \Sigma^N(S_{i,N} * (R_{\mu,N}) / \Sigma^N(|S_{i,N}|)$

- ${\hat R}_{\mu,i}$: 최종적으로 구하고자 하는 점수. 사용자 $\mu$, 아이템 i의 개인화된 예측 평점 값
- $S_{i,N}$: 아이템 i와 가장 유사도가 높은 Top-N개 아이템의 유사도 벡터
- $R_{\mu,N}$: 사용자 $\mu$의 아이템 i와 가장 유사도가 높은 Top-N개 아이템에 대한 실제 평점 벡터